<a href="https://colab.research.google.com/github/KNU-BrainAI-Capstone2022/ITS/blob/main/review_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 38.2 MB/s 
     |████████████████████████████████| 163 kB 72.9 MB/s 
     |████████████████████████████████| 7.6 MB 60.7 MB/s 


In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

import os
import sys
import urllib.request
import re

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
### 한글 데이터 영어로 번역하는 부분  ###

def remove_emoji(st):
    # print(st) 
    only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE)
    return only_BMP_pattern.sub(r'', st)# BMP characters만

def k_to_e(input_st):
    client_id = "---" # 개발자센터에서 발급받은 Client ID 값
    client_secret = "---" # 개발자센터에서 발급받은 Client Secret 값

    input_st = remove_emoji(input_st)

    encText = urllib.parse.quote(input_st)
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        # print(response_body.decode('utf-8'))
        ref = response_body.decode('utf-8')
        found = re.search('"translatedText":"(.+?)","engineType"',ref).group(1)
        return found
        
    else:
        # print("Error Code:" + rescode)
        return "Error Code:" + rescode

# print(k_to_e(sample_review))

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 다른 언어의 데이터면 다른거 쓰기
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # _: embedding vectors of all of the tokens in a sequence
        # pooled_output: embedding vector of [CLS] token, 분류 문제에선 이것만 써도 충분
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

model = BertClassifier()

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# sample_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/knu/Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB, 16GB.xlsx')
sample_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/knu/쿠팡맥북리뷰.xlsx')
sample_data

,상품명,구매자 이름,구매자 평점,리뷰 제목,리뷰 내용,맛 만족도
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",월요병환자,5,m1 m2중 고민이라면 가성비로 추천,⭕ Review with the flow of my money ⭕ Purchase ...,그래픽/전문가용
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",BBOK,5,M1으로 사길 잘했다! 후회없음!,"The release date of Apple 2020 MacBook Air 13,...",사무용
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",보라누나,5,#쌩초보맥북에어#불량검수#체크사항#2D캐드일러포샵스케치,"For the first time in my life, I started the i...",사무용
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",찐찐찐리뷰,5,m2 건너뛰고 m3 살거야,I only do honest reviews!I only do honest revi...,사무용
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",행복사랑해,5,최고의 노트북,#Introduction #Suddenly I needed a 2020 MacBoo...,그래픽/전문가용


In [ ]:
sample_df = sample_data.rename(columns={'상품명':'category', '구매자 평점':'rating', '리뷰 내용':'text_'})
sample_df = sample_df.drop(['구매자 이름', '리뷰 제목', '맛 만족도'], axis=1)

In [ ]:
# sample_df['text_'] = sample_df['text_'].apply(k_to_e)
sample_df['text_'] = sample_df['text_'].apply(remove_emoji)
sample_df['label'] = ['OR']*len(sample_df)
sample_df

,category,rating,text_,label
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,⭕ Review with the flow of my money ⭕ Purchase ...,OR
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",5,"The release date of Apple 2020 MacBook Air 13,...",OR
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",5,"For the first time in my life, I started the i...",OR
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,I only do honest reviews!I only do honest revi...,OR
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,#Introduction #Suddenly I needed a 2020 MacBoo...,OR


In [ ]:
labels = {'CG':0, # 가짜면 0
          'OR':1 # 진짜면 1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']] # df 컬럼 잘 보고 설정
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text_']] # df 컬럼 잘 보고 설정

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
result=[]
def evaluate(model, data):

    test = Dataset(data)

    dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:

        model = model.cuda()
    path = "/content/drive/MyDrive/Colab Notebooks/knu/ClassifierModel.pth"
    model.load_state_dict(torch.load(path))
    total_acc_test = 0
    with torch.no_grad():

        for input, label in dataloader:

              label = label.to(device)
              mask = input['attention_mask'].to(device)
              input_id = input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if(output.argmax(dim=1)[0]==1):
                result.append("OR")
              else:
                result.append("CG")
              
              acc = (output.argmax(dim=1) == label).sum().item()
              total_acc_test += acc
    
    print(result)
    # print(len(data))
    # print(total_acc_test)
    # print(f'Test Accuracy: {total_acc_test / len(data): .3f}')
    # 실제 리뷰 비율
    print('real review ratio : ',result.count('OR')/len(result))
    
 
evaluate(model, sample_df)


['OR', 'OR', 'OR', 'OR', 'OR']
real review ratio :  1.0


In [ ]:
sample_df['predicted_label'] = result
print(sample_df.shape)
sample_df.head()

(5, 5)


,category,rating,text_,label,predicted_label
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,⭕ Review with the flow of my money ⭕ Purchase ...,OR,OR
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",5,"The release date of Apple 2020 MacBook Air 13,...",OR,OR
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",5,"For the first time in my life, I started the i...",OR,OR
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,I only do honest reviews!I only do honest revi...,OR,OR
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,#Introduction #Suddenly I needed a 2020 MacBoo...,OR,OR


In [ ]:
df2 = sample_df.loc[(sample_df['predicted_label'] == 'OR') & (sample_df['rating'] != 3) ]
# print(df2.shape)
df2

,category,rating,text_,label,predicted_label
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,⭕ Review with the flow of my money ⭕ Purchase ...,OR,OR
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",5,"The release date of Apple 2020 MacBook Air 13,...",OR,OR
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",5,"For the first time in my life, I started the i...",OR,OR
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,I only do honest reviews!I only do honest revi...,OR,OR
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,#Introduction #Suddenly I needed a 2020 MacBoo...,OR,OR


In [ ]:
df2['division'] = np.select([df2.rating > 3], [1], default=0)
positive = len(df2.loc[df2['rating'] > 3])
negative = len(df2.loc[df2['rating'] < 3])
print(f'Positive review : ', positive)
print(f'Negative review : ', negative)
df2[0:5]


Positive review :  5
Negative review :  0


,category,rating,text_,label,predicted_label,division
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,⭕ Review with the flow of my money ⭕ Purchase ...,OR,OR,1
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",5,"The release date of Apple 2020 MacBook Air 13,...",OR,OR,1
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",5,"For the first time in my life, I started the i...",OR,OR,1
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,I only do honest reviews!I only do honest revi...,OR,OR,1
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,#Introduction #Suddenly I needed a 2020 MacBoo...,OR,OR,1


In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df2):

        self.divisions = [division for division in df2['division']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df2['text_']]
    def classes(self):
        return self.divisions

    def __len__(self):
        return len(self.divisions)

    def get_batch_divisions(self, idx):
        # Fetch a batch of labels
        return np.array(self.divisions[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_divisions(idx)

        return batch_texts, batch_y

df2[:5]

,category,rating,text_,label,predicted_label,division
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,⭕ Review with the flow of my money ⭕ Purchase ...,OR,OR,1
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",5,"The release date of Apple 2020 MacBook Air 13,...",OR,OR,1
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",5,"For the first time in my life, I started the i...",OR,OR,1
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,I only do honest reviews!I only do honest revi...,OR,OR,1
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,#Introduction #Suddenly I needed a 2020 MacBoo...,OR,OR,1


In [ ]:
ddf = df2
ddf = ddf.drop('text_',axis=1)
ddf['text_'] = ['Good','amazing','the best','i love it','excellent']
ddf

,category,rating,label,predicted_label,division,text_
0,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,OR,OR,1,Good
1,"Apple 2020 맥북 에어 13, 실버, M1, M1 8 core, 512GB,...",5,OR,OR,1,amazing
2,"Apple 2020 맥북 에어 13, 스페이스 그레이, M1, M1 7 core, ...",5,OR,OR,1,the best
3,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,OR,OR,1,i love it
4,"Apple 2020 맥북 에어 13, 실버, M1, M1 7 core, 256GB,...",5,OR,OR,1,excellent


In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()
    checking_num1 = 0
    checking_num2 = 0
    checking_num3 = 0
    checking_num4 = 0
    path = "/content/drive/MyDrive/Colab Notebooks/knu/NetModel_s.pth"
    model.load_state_dict(torch.load(path))    
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_division in test_dataloader:

              test_division = test_division.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if output.argmax(dim=1)[0] == 1:
                  checking_num1 +=1
              elif output.argmax(dim=1)[0] == 0:
                  checking_num3 +=1              
              else:
                  checking_num4 +=1
              checking_num2+=1
              acc = (output.argmax(dim=1) == test_division).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print('==',total_acc_test)
    print('==',len(test_data))
    print(checking_num1)
    print(checking_num2)
    print(checking_num3)
    print(checking_num4)
    print(f'Positive review : ', checking_num1)
    print(f'Total review : ', checking_num2)
    print(f'Positive review rate: {(checking_num1/checking_num2)*100: .3f}')
evaluate(model, ddf)

Test Accuracy:  1.000
Test Accuracy:  1.000
== 5
== 5
5
5
0
0
Positive review :  5
Total review :  5
Positive review rate:  100.000


In [ ]:
evaluate(model, df2)

Test Accuracy:  0.000
Test Accuracy:  0.000
== 0
== 5
0
5
5
0
Positive review :  0
Total review :  5
Positive review rate:  0.000
